In [2]:
pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


In [4]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from dash.exceptions import PreventUpdate


In [ ]:
app = dash.Dash(__name__)

metadata_file = "metadata.csv"
metadata_df = pd.read_csv(metadata_file)
metadata_df

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
0,discharge,[2010. 7. 21. 15. 0. ...,4,B0047,0,1,00001.csv,1.6743047446975208,NaN,NaN
1,impedance,[2010. 7. 21. 16. 53. ...,24,B0047,1,2,00002.csv,NaN,0.05605783343888099,0.20097016584458333
2,charge,[2010. 7. 21. 17. 25. ...,4,B0047,2,3,00003.csv,NaN,NaN,NaN
3,impedance,[2010 7 21 20 31 5],24,B0047,3,4,00004.csv,NaN,0.05319185850921101,0.16473399914864734
4,discharge,[2.0100e+03 7.0000e+00 2.1000e+01 2.1000e+01 2...,4,B0047,4,5,00005.csv,1.5243662105099023,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7560,impedance,[2010. 9. 30. 7. 36. ...,24,B0055,247,7561,07561.csv,NaN,0.0968087979207628,0.15489738203707232
7561,discharge,[2010. 9. 30. 8. 8. ...,4,B0055,248,7562,07562.csv,1.0201379996149256,NaN,NaN
7562,charge,[2010. 9. 30. 8. 48. 54.25],4,B0055,249,7563,07563.csv,NaN,NaN,NaN
7563,discharge,[2010. 9. 30. 11. 50. ...,4,B0055,250,7564,07564.csv,0.9907591663373165,NaN,NaN


In [ ]:
print(metadata_df.info())
print(metadata_df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7565 entries, 0 to 7564
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   type                 7565 non-null   object
 1   start_time           7565 non-null   object
 2   ambient_temperature  7565 non-null   int64 
 3   battery_id           7565 non-null   object
 4   test_id              7565 non-null   int64 
 5   uid                  7565 non-null   int64 
 6   filename             7565 non-null   object
 7   Capacity             2794 non-null   object
 8   Re                   1956 non-null   object
 9   Rct                  1956 non-null   object
dtypes: int64(3), object(7)
memory usage: 591.1+ KB
None
type                      0
start_time                0
ambient_temperature       0
battery_id                0
test_id                   0
uid                       0
filename                  0
Capacity               4771
Re                     5

In [ ]:
metadata_df['Capacity'] = pd.to_numeric(metadata_df['Capacity'], errors='coerce')
capacity_mean = metadata_df['Capacity'].mean()
metadata_df['Capacity'] = metadata_df['Capacity'].fillna(capacity_mean)

metadata_df['Re'] = pd.to_numeric(metadata_df['Re'], errors='coerce')
metadata_df['Rct'] = pd.to_numeric(metadata_df['Rct'], errors='coerce')

re_mean = metadata_df['Re'].mean()
rct_mean = metadata_df['Rct'].mean()
metadata_df['Re'] = metadata_df['Re'].fillna(re_mean)
metadata_df['Rct'] = metadata_df['Rct'].fillna(rct_mean)


In [ ]:
print(metadata_df.info())
print(metadata_df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7565 entries, 0 to 7564
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   type                 7565 non-null   object 
 1   start_time           7565 non-null   object 
 2   ambient_temperature  7565 non-null   int64  
 3   battery_id           7565 non-null   object 
 4   test_id              7565 non-null   int64  
 5   uid                  7565 non-null   int64  
 6   filename             7565 non-null   object 
 7   Capacity             7565 non-null   float64
 8   Re                   7565 non-null   float64
 9   Rct                  7565 non-null   float64
dtypes: float64(3), int64(3), object(4)
memory usage: 591.1+ KB
None
type                   0
start_time             0
ambient_temperature    0
battery_id             0
test_id                0
uid                    0
filename               0
Capacity               0
Re                     0

In [ ]:
print(metadata_df.describe())


       ambient_temperature      test_id          uid     Capacity  \
count          7565.000000  7565.000000  7565.000000  7565.000000   
mean             20.017713   176.012558  3783.000000     1.326543   
std              11.082914   152.174147  2183.971726     0.285841   
min               4.000000     0.000000     1.000000     0.000000   
25%               4.000000    55.000000  1892.000000     1.326543   
50%              24.000000   129.000000  3783.000000     1.326543   
75%              24.000000   255.000000  5674.000000     1.326543   
max              44.000000   615.000000  7565.000000     2.640149   

                 Re           Rct  
count  7.565000e+03  7.565000e+03  
mean  -4.976500e+11  1.055903e+12  
std    1.113788e+13  2.363212e+13  
min   -9.689245e+14 -2.091081e+02  
25%   -4.976500e+11  2.405557e-01  
50%   -4.976500e+11  1.055903e+12  
75%    4.033353e-02  1.055903e+12  
max    4.482291e+02  2.055843e+15  


In [ ]:
from sklearn.preprocessing import StandardScaler

# Select features for modeling
features = metadata_df[['ambient_temperature', 'Re', 'Rct']]

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

metadata_df[['ambient_temperature_scaled', 'Re_scaled', 'Rct_scaled']] = scaled_features

In [ ]:
metadata_df

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct,ambient_temperature_scaled,Re_scaled,Rct_scaled
0,discharge,[2010. 7. 21. 15. 0. ...,4,B0047,0,1,00001.csv,1.674305,-4.976500e+11,1.055903e+12,-1.445357,5.480323e-18,-5.165782e-18
1,impedance,[2010. 7. 21. 16. 53. ...,24,B0047,1,2,00002.csv,1.326543,5.605783e-02,2.009702e-01,0.359341,4.468380e-02,-4.468380e-02
2,charge,[2010. 7. 21. 17. 25. ...,4,B0047,2,3,00003.csv,1.326543,-4.976500e+11,1.055903e+12,-1.445357,5.480323e-18,-5.165782e-18
3,impedance,[2010 7 21 20 31 5],24,B0047,3,4,00004.csv,1.326543,5.319186e-02,1.647340e-01,0.359341,4.468380e-02,-4.468380e-02
4,discharge,[2.0100e+03 7.0000e+00 2.1000e+01 2.1000e+01 2...,4,B0047,4,5,00005.csv,1.524366,-4.976500e+11,1.055903e+12,-1.445357,5.480323e-18,-5.165782e-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7560,impedance,[2010. 9. 30. 7. 36. ...,24,B0055,247,7561,07561.csv,1.326543,9.680880e-02,1.548974e-01,0.359341,4.468380e-02,-4.468380e-02
7561,discharge,[2010. 9. 30. 8. 8. ...,4,B0055,248,7562,07562.csv,1.020138,-4.976500e+11,1.055903e+12,-1.445357,5.480323e-18,-5.165782e-18
7562,charge,[2010. 9. 30. 8. 48. 54.25],4,B0055,249,7563,07563.csv,1.326543,-4.976500e+11,1.055903e+12,-1.445357,5.480323e-18,-5.165782e-18
7563,discharge,[2010. 9. 30. 11. 50. ...,4,B0055,250,7564,07564.csv,0.990759,-4.976500e+11,1.055903e+12,-1.445357,5.480323e-18,-5.165782e-18


In [ ]:
import dash
from dash import dcc, html
import plotly.express as px

# Create a Dash application
app = dash.Dash(__name__)

# Sample Plotly plot (scatter plot)
fig = px.scatter(metadata_df, x='ambient_temperature', y='Capacity', title="Ambient Temperature vs Capacity")

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Battery Data Dashboard"),
    dcc.Graph(id='scatter-plot', figure=fig),
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:


app = dash.Dash(__name__)

X = metadata_df[['ambient_temperature_scaled', 'Re_scaled', 'Rct_scaled']]
y = metadata_df['Capacity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn_model = KNeighborsRegressor(n_neighbors=10, weights='uniform', metric='euclidean')
knn_model.fit(X_train_scaled, y_train)

y_pred = knn_model.predict(X_test_scaled)

metadata_df['predicted_capacity'] = knn_model.predict(scaler.transform(X))

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Final Model MSE:", mse)
print("Final Model R²:", r2)

Final Model MSE: 0.06140122044996822
Final Model R²: 0.22388127927006296


In [ ]:
fig1 = px.scatter(metadata_df, x='ambient_temperature', y='Capacity', title="Ambient Temperature vs Capacity")
fig2 = px.scatter(metadata_df, x='ambient_temperature', y='predicted_capacity', title="Predicted Capacity vs Ambient Temperature", color='predicted_capacity')
fig3 = px.line(metadata_df, x='ambient_temperature', y=['Capacity', 'predicted_capacity'], title="Actual vs Predicted Capacity")
fig4 = px.bar(metadata_df, x='ambient_temperature', y=['Capacity', 'predicted_capacity'], title="Capacity Comparison", barmode='group')
fig5 = px.histogram(metadata_df, x='ambient_temperature', nbins=30, title="Distribution of Ambient Temperature")
fig6 = px.histogram(metadata_df, x='Capacity', nbins=30, title="Distribution of Capacity")
fig7 = px.imshow(metadata_df[['ambient_temperature_scaled', 'Re_scaled', 'Rct_scaled', 'Capacity']].corr(), title="Correlation Heatmap")
fig8 = px.box(metadata_df, y='Capacity', title="Box Plot of Capacity")
fig9 = px.violin(metadata_df, y='Capacity', box=True, points="all", title="Violin Plot of Capacity")
fig10 = px.scatter_3d(metadata_df, x='ambient_temperature', y='Re_scaled', z='Rct_scaled', color='Capacity', title="3D Scatter Plot of Features")

# Handling layout with multiple plots
app.layout = html.Div([
    html.H1("Battery Data Dashboard"),

    # Scatter and Line Plots
    html.Div([
        dcc.Graph(id='scatter-plot', figure=fig1),
        dcc.Graph(id='scatter-predicted', figure=fig2)
    ], style={'display': 'flex', 'flex-direction': 'row', 'justify-content': 'space-between'}),

    # Actual vs Predicted Capacity Line Plot
    html.Div([
        dcc.Graph(id='line-plot', figure=fig3),
    ], style={'margin-top': '20px'}),

    # Bar Plot and Histograms
    html.Div([
        dcc.Graph(id='bar-plot', figure=fig4),
    ], style={'margin-top': '20px'}),

    html.Div([
        dcc.Graph(id='histogram-temp', figure=fig5),
        dcc.Graph(id='histogram-capacity', figure=fig6)
    ], style={'display': 'flex', 'flex-direction': 'row', 'justify-content': 'space-between', 'margin-top': '20px'}),

    # Heatmap and Distribution Plots
    html.Div([
        dcc.Graph(id='heatmap', figure=fig7),
    ], style={'margin-top': '20px'}),

    html.Div([
        dcc.Graph(id='box-plot', figure=fig8),
        dcc.Graph(id='violin-plot', figure=fig9)
    ], style={'display': 'flex', 'flex-direction': 'row', 'justify-content': 'space-between', 'margin-top': '20px'}),

    # 3D Scatter Plot and Pairwise Scatter Plot
    html.Div([
        dcc.Graph(id='3d-scatter', figure=fig10),
    ], style={'margin-top': '20px'}),



    # Display Model Performance
    html.Div([
        html.H3(f"Mean Squared Error: {mse:.4f}"),
        html.H3(f"R² Score: {r2:.4f}")
    ])
])

# Run the app
if __name__ == '__main__':
    try:
        app.run_server(debug=True)
    except Exception as e:
        print(f"Error running server: {e}")